<a href="https://colab.research.google.com/github/gimhanul/Datathon2021/blob/master/findGoodRandomState.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 테이블 형식의 데이터를 처리하는 pandas 라이브러리를 임포트
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn import svm

model_dic = {
    "SVM" : svm.SVR(kernel='rbf', C = 6.25, gamma=1.1)
}

In [ ]:
# 8개 컬럼의 수치 데이터를 정규화 하여 학습 가능한 형태로 재가공 합니다.
# 다른 정규화 방식을 찾아서 적용해보고 정규화 방식에 따라 결과가 어떻게 달라지는지 확인해봅니다.
from sklearn.preprocessing import MinMaxScaler 

# 학습 및 평가 데이터를 DataFrame 형태로 입력받아 정규화된 데이터로 변환하는 함수 
def scale(x_train, x_test):
    columns = x_train.columns

    scaler = MinMaxScaler()

    # 학습 데이터의 age ~ broad jump_cm 열에 해당하는 값을 가져와 피팅 시킵니다.
    scaler.fit(x_train)

    # 피팅된 scaler를 기준으로 학습데이터를 정규화합니다.
    x_train = scaler.transform(x_train)
    # 평가 데이터도 마찬가지로 학습 데이터로 피팅된 scaler를 기준으로 정규화를 진행합니다.
    x_test = scaler.transform(x_test)

    # DataFrame 형태로 변환하여 반환합니다.
    x_train = pd.DataFrame(columns=columns, data=x_train)
    x_test = pd.DataFrame(columns=columns, data=x_test)

    return x_train, x_test


In [ ]:
# train.csv, test.csv 파일을 DataFrame 형태로 읽어옵니다.
def findGoodRandomState(state):
  train_df = pd.read_csv('/content/drive/MyDrive/datathon/dataset/train.csv')
  test_df = pd.read_csv('/content/drive/MyDrive/datathon/dataset/test.csv')


  y_train = train_df.pop('body fat_%')

  train_df['gender'] = train_df['gender'].map({'M':0, 'F':1})
  test_df['gender'] = test_df['gender'].map({'M':0, 'F':1})

  #정규화를 위해서 gender field 분리
  train_gtemp = train_df.pop('gender')
  test_gtemp = test_df.pop('gender')

  x_train, x_test = scale(train_df, test_df)

  x_train['gender'] = train_gtemp
  x_test['gender'] = test_gtemp

  x_train['gender'] = x_train['gender'].astype('bool')
  x_test['gender'] = x_test['gender'].astype('bool')

  x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, random_state=state)

  model_p = {}

  model_dic["SVM"].fit(x_train, y_train)
  model_p["SVM"] = model_dic["SVM"].predict(x_valid)

  model_pdf = pd.DataFrame(model_p)
  model_pdf['valid'] = list(y_valid)

  model_mae = {}

  mae = mean_absolute_error(model_pdf['valid'], model_p["SVM"])
  model_mae["SVM"] = mae
  print(f'{state} {mae}')
  return mae

In [ ]:
min = 0
nun = 0.00
for i in range(105000,10000000):
  t = findGoodRandomState(i)
  if t < nun:
    nun = t
    min = i

print('min = ', min)

105000 3.8594785448157314
105001 3.8534576810591417
105002 3.9607783096052724
105003 3.849686925437652
105004 3.8040433262134417
105005 3.8071666950371665
105006 3.864038168728114
105007 3.8322720146519407
105008 3.7842404893817787
105009 3.823682095862665
105010 3.8039330816232724
105011 3.8653154924357924
105012 3.7652721688127104
105013 3.798545149654314
105014 3.8424191804283496
105015 3.881540729984943
105016 3.8959143428602387
105017 3.7782544830088645
105018 3.761867436610436
105019 3.8951095861148373
105020 3.7727604618529287
105021 3.769241751203992
105022 3.9190465370856495
105023 3.9135417183063246
105024 3.793930162147891
105025 3.812213458480005
105026 3.808199650865678
105027 3.87644179969207
105028 3.954460477918151
105029 3.8362883340755536
105030 3.7832656045644217
105031 3.8797779181779215
105032 3.8093418330882405
105033 3.842027743736476
105034 3.9132193238998974
105035 3.783832681929834
105036 3.811000065430283
105037 3.9364227612878935
105038 3.836212134212108
105

# 모델평가

In [ ]:
from sklearn.metrics import mean_absolute_error
import numpy as np

# 평가 데이터 예측 및 제출 파일 생성

In [ ]:
# 검증셋 예측과 동일하게 평가 데이터 예측을 진행합니다.
preds = model_dic['SVM'].predict(x_test)
print(preds.shape)

(4018,)


In [ ]:
# 제출파일 예시 파일을 읽어와 predict 열에 평가셋 예측값을 기록하여 저장합니다.
submit = pd.read_csv('/content/drive/MyDrive/datathon/dataset/submit_sample.csv')
submit['predict'] = preds

# 제출 파일 확인
submit

,predict
0,23.320110
1,22.345531
2,16.025266
3,19.500507
4,36.720476
...,...
4013,19.987955
4014,18.787552
4015,26.750057
4016,17.904372


In [ ]:
# 저장
submit.to_csv('/content/drive/MyDrive/datathon/dataset/submit.csv', index=False)